# Intro to Writing a Pipeline

author

last run successfully

In [1]:
#imports
import os
import numpy as np

# TODO look into these
from rail.core.utils import RAILDIR
#from rail.estimation.algos.bpz_version.utils import RAIL_BPZ_DIR

from rail.core import common_params

# TODO how to use
import rail.stages
rail.stages.import_and_attach_all()
from rail.stages import *


#from rail.core.data import TableHandle
#from rail.core.stage import RailStage

Imported rail.hub
Imported rail.astro_tools
Imported rail.core
Imported rail.stages
Imported rail.bpz
Imported rail.cmnn
Imported rail.delight
Failed to import rail.dsps because: You need to have the SPS_HOME environment variable
Imported rail.flexzboost
Failed to import rail.gpz because: No module named 'rail.estimation.algos.gpz_v1'
Imported rail.pipelines
Failed to import rail.pzflow because: No module named 'rail.estimation.algos.pzflow'
Imported rail.sklearn
Imported rail.som
Attached 10 base classes and 46 fully formed stages to rail.stages


### Set directory paths
early on set RAILDIR, BPZDIR so you can find configs shipped with the packages (artifact of directory differences between the ez install vs from source)

In [2]:
RAIL_BPZ_DIR = "/Users/orl/code/DESC-RAIL/rail_bpz/src"
# TODO is there an easy way to get bpz dir programmatically? or at least, remember to make this generic before push

### Catalog-specific settings

defining stuff thats diff from hsc vs dc2, things that are different for the catalog

to go between catalogs, you “just have to change the first two cells” (this was while in estimate_all_hsc?) → changing names of the bands & changing common_params (and also the input file)

this is hsc stuff:

In [3]:
bands = 'grizy'
maglims = [27.66, 27.25, 26.6, 26.24, 25.35]
maglim_dict={}
for band,limx in zip(bands, maglims):
    maglim_dict[f"HSC{band}_cmodel_dered"] = limx

### Setting common parameters to all the stages in the estimation

setting the defaults for the individual parameters, before making any stages

when you make a stage, it copies the current common_params so if you change it elsewhere it stays the same for the stage

In [4]:
common_params.set_param_defaults(
    bands=[f'HSC{band}_cmodel_dered' for band in bands],
    err_bands=[f'{band}_cmodel_magerr' for band in bands],
    nondetect_val=np.nan,
    ref_band='HSCi_cmodel_dered',
    redshift_col='specz_redshift',
    mag_limits=maglim_dict,
    zmax = 6.0,
)

### Set datastore

explain datastore briefly, then reference datastore notebook

In [5]:

DS = RailStage.data_store
DS.__class__.allow_overwrite = True

at some point mention:
- dir(pipe)
- and other useful things, but reference utilities nb? or keep here 



is there a way to list out all possible stages we could add?

from RAIL_estimation_demo:

> Getting the list of available Estimators

RailStage knows about all of the sub-types of stages.  The are stored in the `RailStage.pipeline_stages` dict.  By looping through the values in that dict we can and asking if each one is a sub-class of `rail.estimation.estimator.CatEstimator` we can identify the available estimators that operator on catalog-like inputs.

Note: after my talk with Eric -> it's important to mention that these will only identify stages that you've imported. so if you're doing the import and attach import style, they'll all be there, but if you're importing indiviudal stages, only some will be added to the dict. Eric says this is primarily used for debugging, though the fact that Sam included this in a notebook in a more human-readable exploration-oriented way does feel like this is a valid use as well.

In [6]:
for key in RailStage.pipeline_stages:
    if 'engines' in str(RailStage.pipeline_stages[key][0]):
        print(f'{key} : {RailStage.pipeline_stages[key][0]}')

FlowModeler : <class 'rail.creation.engines.flowEngine.FlowModeler'>
FlowCreator : <class 'rail.creation.engines.flowEngine.FlowCreator'>
FlowPosterior : <class 'rail.creation.engines.flowEngine.FlowPosterior'>
